# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
Arxiv has 39 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.12713


extracting tarball to tmp_2408.12713... done.
Retrieving document from  https://arxiv.org/e-print/2408.12765


extracting tarball to tmp_2408.12765...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.12854


...

M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']


Found 75 bibliographic references in tmp_2408.12854/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2408.12929
extracting tarball to tmp_2408.12929... done.
Retrieving document from  https://arxiv.org/e-print/2408.13141


extracting tarball to tmp_2408.13141...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2713/1211882699.py:51: LatexWarning: 2408.13141 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12854-b31b1b.svg)](https://arxiv.org/abs/2408.12854) | **Exploring Active Galactic Nuclei and Little Red Dots with the Obelisk simulation**  |
|| M. Volonteri, et al. -- incl., <mark>M. Habouzit</mark> |
|*Appeared on*| *2024-08-26*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            The James Webb Space telescope has discovered an abundant population of broad line emitters, typical signposts for Active Galactic Nuclei (AGN). Many of these sources have red colors and a compact appearance that has led to naming them `Little Red Dots'. In this paper we develop a detailed framework to estimate the photometry of AGN embedded in galaxies extracted from the Obelisk cosmological simulation to understand the properties of color-selected Little Red Dots (cLRDs) in the context of the full AGN and massive black hole population. We find that using realistic spectral energy distributions (SEDs) and attenuation for AGN we can explain the shape of the cLRD SED as long as galaxies host a sufficiently luminous AGN that is not too much or too little attenuated. When attenuation is too low or too high, AGN do not enter the cLRD selection, because the AGN dominates over the host galaxy too much in blue filters, or it does not contribute to photometry anywhere, respectively. cLRDs are also characterized by high Eddington ratios, possibility super-Eddington, and/or high ratios between black hole and stellar mass.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12713-b31b1b.svg)](https://arxiv.org/abs/2408.12713) | **NEXUS: the North ecliptic pole EXtragalactic Unified Survey**  |
|| Y. Shen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-08-26*|
|*Comments*| *19 pages*|
|**Abstract**|            NEXUS is a JWST Multi-Cycle (Cycles 3-5; 368 primary hrs) GO Treasury imaging and spectroscopic survey around the North Ecliptic Pole. It contains two overlapping tiers. The Wide tier ($\sim 400~{\rm arcmin}^2$) performs NIRCam/WFSS 2.4-5 micron grism spectroscopy with three epochs over 3 years (final continuum ${\rm S/N/pixel>3}$ at F444W$<22.2$). The Deep tier ($\sim 50~{\rm arcmin}^2$) performs high-multiplexing NIRSpec 0.6-5.3 micron MOS/PRISM spectroscopy for $\sim 10,000$ targets, over 18 epochs with a 2-month cadence (epoch/final continuum ${\rm S/N/pixel>3}$ at F200W$\lesssim 27/29$). All epochs have simultaneous multi-band NIRCam and MIRI imaging ($5\sigma$ final depths of $\sim 28-29$ in NIRCam and $\sim 25$ in MIRI). The field is within the continuous viewing zone of JWST, and is fully covered by the Euclid Ultra-Deep Field, with 0.9-2 micron deep Euclid spectroscopy and cadenced photometry. NEXUS has three science pillars. First, with its massive and nearly complete (flux-limited) spectroscopic samples and deep photometry, it will perform efficient classification and physical characterization of galaxies and AGNs from $z\sim 1$ to Cosmic Dawn. With the large contiguous area coverage, it will measure the spatial clustering and demography of the first galaxies and SMBHs at $z>6$. Second, multi-epoch observations enable systematic time-domain investigations, focusing on $z>3$ transients and low-mass AGN reverberation mapping. Third, the comprehensive data set will enable knowledge transfer to other legacy fields, create data challenges, and initiate benchmark work for future space missions. With rapid public releases of processed data and an open invitation for collaboration, NEXUS aims for broad and swift community engagement, to become a powerhouse to drive transformative advancements in multiple key science areas of astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12765-b31b1b.svg)](https://arxiv.org/abs/2408.12765) | **Variable Stars in M31 Stellar Clusters from the Panchromatic Hubble Andromeda Treasury**  |
|| R. Smith, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-08-26*|
|*Comments*| *36 pages, 18 figures; accepted for publication in ApJ*|
|**Abstract**|            Variable stars in stellar clusters can offer key constraints on stellar evolution and pulsation models, utilising estimates of host cluster properties to constrain stellar physical parameters. We present a catalogue of 86 luminous (F814W<19) variable stars in M31 clusters identified by mining the archival Panchromatic Hubble Andromeda Treasury (PHAT) survey using a combination of statistical analysis of sparse PHAT light curves and difference imaging. We determine the evolutionary phases and initial masses of these variable stars by matching them with theoretical isochrones generated using host cluster properties from the literature. We calculate the probability of PHAT photometry being blended due to the highly crowded nature of cluster environments for each cluster-variable star, using these probabilities to inform our level of confidence in the derived properties of each star. Our 86 cluster-variable stars have initial masses between 0.8--67 $M_{\odot}$. Their evolutionary phases span the main sequence, more evolved hydrogen- and helium-burning phases, and the post-asymptotic giant branch. We identify numerous candidate variable star types: RV Tauri variables, red supergiants and slowly pulsating B-type supergiants, along with Wolf Rayet stars, $\alpha$ Cygni and Mira variables, a classical Cepheid and a possible super-asymptotic giant. We characterise 12 cluster-variable stars at higher confidence based on their difference image quality and lower blending probability. Ours is the first systematic study of variable stars in extragalactic stellar clusters leveraging the superior resolution of the Hubble Space Telescope and demonstrating the unique power of stellar clusters in constraining the fundamental properties of variable stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12929-b31b1b.svg)](https://arxiv.org/abs/2408.12929) | **The Velocity Aberration Effect of the CSST Main Survey Camera**  |
|| H.-M. Feng, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-08-26*|
|*Comments*| *13 pages, 8 figures; accepted by RAA*|
|**Abstract**|            In this study, we conducted simulations to find the geometric aberrations expected for images taken by the Main Survey Camera (MSC) of the Chinese Space Station Telescope (CSST) due to its motion. As anticipated by previous work, our findings indicate that the geometric distortion of light impacts the focal plane's apparent scale, with a more pronounced influence as the size of the focal plane increases. Our models suggest that the effect consistently influences the pixel scale in both the vertical and parallel directions. The apparent scale variation follows a sinusoidal distribution throughout one orbit period. Simulations reveal that the effect is particularly pronounced in the center of the Galaxy and gradually diminishes along the direction of ecliptic latitude. At low ecliptic latitudes, the total aberration leads to about 0.94 pixels offset (a 20-minute exposure) and 0.26 pixels offset (a 300-second exposure) at the edge of the field of view, respectively. Appropriate processings for the geometric effect during the CSST pre- and post-observation phases are presented.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.13141-b31b1b.svg)](https://arxiv.org/abs/2408.13141) | **Laser-induced fragmentation of coronene cations**  |
|| S. Panchagnula, et al. -- incl., <mark>J. Bouwman</mark> |
|*Appeared on*| *2024-08-26*|
|*Comments*| *17 pages, 12 figure. Version submitted after referee report to PCCP. Supplementary information available at this https URL*|
|**Abstract**|            Polycyclic aromatic hydrocarbons are an important component of the interstellar medium of galaxies and photochemistry plays a key role in the evolution of these species in space. Here, we explore the photofragmentation behaviour of the coronene cation (C24H12+) using time of flight mass spectrometry. The experiments show photodissociation fragmentation channels including the formation of bare carbon clusters (Cn+) and hydrocarbon chains (CnHx+). The mass spectrum of coronene is dominated by peaks from C11+ and C7H+. Density functional theory was used to calculate relative energies, potential dissociation pathways, and possible structures for relevant species. We identify 6-6 to 5-7 ring isomerisation as a key step in the formation of both the bare carbon clusters and the hydrocarbon chains observed in this study. We present the dissociation mechanism outlined here as a potential formation route for C60 and other astrochemically relevant species.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.12854/./Figures/NHdistr_z6_LRDs.png', 'tmp_2408.12854/./Figures/NH_taugal.png', 'tmp_2408.12854/./Figures/mag_sEdd_mag_SMC.png', 'tmp_2408.12854/./Figures/mag_pac_mag_SMC.png', 'tmp_2408.12854/./Figures/mag_sEdd_mag_MW.png', 'tmp_2408.12854/./Figures/mag_pac_mag_MW.png', 'tmp_2408.12854/./Figures/bolLF_LRDs_Obelisk_gal.png']
copying  tmp_2408.12854/./Figures/NHdistr_z6_LRDs.png to _build/html/
copying  tmp_2408.12854/./Figures/NH_taugal.png to _build/html/
copying  tmp_2408.12854/./Figures/mag_sEdd_mag_SMC.png to _build/html/
copying  tmp_2408.12854/./Figures/mag_pac_mag_SMC.png to _build/html/
copying  tmp_2408.12854/./Figures/mag_sEdd_mag_MW.png to _build/html/
copying  tmp_2408.12854/./Figures/mag_pac_mag_MW.png to _build/html/
copying  tmp_2408.12854/./Figures/bolLF_LRDs_Obelisk_gal.png to _build/html/
exported in  _build/html/2408.12854.md
    + _build/html/tmp_2408.12854/./Figures/NHdistr_z6_LRDs.png
    + _build/html/tmp_2408.12854/./Figures/NH_taugal.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mv}[1]{{\textcolor{orange}{\sf{[MV: #1]}} }}$
$\newcommand{\yo}[1]{{\textcolor{purple}{\sf{[YD: #1]}} }}$
$\newcommand{\ca}[1]{{\textcolor{brown}{\sf{[CA: #1]}} }}$
$\newcommand{\mt}[1]{{\textcolor{blue}{\sf{[MT: #1]}} }}$
$\newcommand{\mh}[1]{{\textcolor{teal}{\sf{[MH: #1]}} }}$
$\newcommand{\rb}[1]{{\textcolor{magenta}{\sf{[RB: #1]}} }}$
$\newcommand{\Obelisk}{\textsc{Obelisk}}$
$\newcommand{\fedd}{\ensuremath{f_{\rm Edd}}}$
$\newcommand{\kms}{ {\rm km s^{-1}}}$
$\newcommand{\msun}{ {\rm M_\odot}}$
$\newcommand{\mstar}{\ensuremath{M_{\rm star}}}$
$\newcommand{\mbh}{\ensuremath{M_{\rm BH}}}$
$\newcommand{\yilun}[1]{{\textcolor{cyan}{\sf{[Yilun: #1]}} }}$
$\newcommand{\al}[1]{{\textcolor{magenta}{\sf{[AL: #1]}} }}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$</div>



<div id="title">

# Exploring Active Galactic Nuclei and Little Red Dots with the Obelisk simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.12854-b31b1b.svg)](https://arxiv.org/abs/2408.12854)<mark>Appeared on: 2024-08-26</mark> -  _Submitted to A&A_

</div>
<div id="authors">

M. Volonteri, et al. -- incl., <mark>M. Habouzit</mark>

</div>
<div id="abstract">

**Abstract:** The James Webb Space telescope has discovered an abundant population of broad line emitters, typical signposts for Active Galactic Nuclei (AGN). Many of these sources have red colors and a compact appearance that has led to naming them `Little Red Dots'. In this paper we develop a detailed framework to estimate the photometry of AGN embedded in galaxies extracted from the $\Obelisk$ cosmological simulation to understand the properties of color-selected Little Red Dots (cLRDs) in the context of the full AGN and massive black hole population. We find that using realistic spectral energy distributions (SEDs) and attenuation for AGN we can explain the shape of the cLRD SED as long as galaxies host a sufficiently luminous AGN that is not too much or too little attenuated. When attenuation is too low or too high, AGN do not enter the cLRD selection, because the AGN dominates over the host galaxy too much in blue filters, or it does not contribute to photometry anywhere, respectively. cLRDs are also characterized by high Eddington ratios, possibility super-Eddington, and/or high ratios between black hole and stellar mass.

</div>

<div id="div_fig1">

<img src="tmp_2408.12854/./Figures/NHdistr_z6_LRDs.png" alt="Fig4.1" width="50%"/><img src="tmp_2408.12854/./Figures/NH_taugal.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Top: distribution of hydrogen column density seen by the AGN as a function of galaxy stellar mass at $z=6$. This is the column density integrated above the dust sublimation radius, i.e. the column density that enters into calculating dust absorption. The red points highlight the cases that host a cLRD meeting the UNCOVER selection. Bottom: distribution of `effective' hydrogen column density for starlight obtained from the ratio of intrinsic and attenuated fluxes in each JWST filter we have considered (Eq. \ref{eq:effNH}). (*fig:NHdistr*)

</div>
<div id="div_fig2">

<img src="tmp_2408.12854/./Figures/mag_sEdd_mag_SMC.png" alt="Fig9.1" width="25%"/><img src="tmp_2408.12854/./Figures/mag_pac_mag_SMC.png" alt="Fig9.2" width="25%"/><img src="tmp_2408.12854/./Figures/mag_sEdd_mag_MW.png" alt="Fig9.3" width="25%"/><img src="tmp_2408.12854/./Figures/mag_pac_mag_MW.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Magnitudes for the `sEdd' and `overm' models and SMC-like dust (top panels) and MW-like dust (bottom panels).  The orange triangles and gray circles show separately the flux of AGN and galaxies, when selecting systems with $L_{\rm bol}>10^{44}  {\rm erg   s}^{-1}$ and m444W$<$28 (total flux, summing both AGN and galaxy contribution). The red dots have both of the above, plus the UNCOVER color selection, and show the combined galaxy+AGN flux. We have limited the y-axis to 50 to highlight the relevant parameter space. At 115 and 200 the galaxy is almost always brighter than the AGN, whose contribution increases as the wavelength increases. For the `overm' case some AGN are brighter than the galaxy also at short wavelenghts.  (*fig:mag_z6_80*)

</div>
<div id="div_fig3">

<img src="tmp_2408.12854/./Figures/bolLF_LRDs_Obelisk_gal.png" alt="Fig2" width="100%"/>

**Figure 2. -** Bolometric luminosity function for the actual MBHs in the simulation (`true') and for the models describes in  Table \ref{tab:models}, where "resc" means we have corrected the number densities for the overdensity bias: $\Obelisk$  is a protocluster. To obtain the cosmic average we have rescaled the $\Obelisk$  galaxy mass function to that of the NewHorizon simulation, which simulates an average region of the Universe. This figure assumes an active fraction of unity, and therefore represents an upper limit to the LF for each of the models. The results are compared to the luminosity functions derived in \citet[][M+24]{2024ApJ...963..129M} and \citet[][G+24]{2024ApJ...964...39G} from JWST data and to fits to the evolution of the bolometric luminosity function derived in \citet[][S+20, pre-JWST]{2020MNRAS.495.3252S}.  (*fig:bolLF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.12854"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

167  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
